#### Initialize

In [48]:
from openai import OpenAI
client = OpenAI()

#### Pretty Printing Helper


In [49]:
import json

def print_json(obj):
    display(json.loads(obj.model_dump_json()))

#### Retrieve Assistant

In [50]:
home0001_assistant = client.beta.assistants.retrieve("asst_1XlpF0EPSnscirf5ZfHWwSLc") 
print_json(home0001_assistant)

{'id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
 'created_at': 1732627992,
 'description': None,
 'instructions': "\nYou are an expert customer service rep for the housing collective HOME0001. Use your knowledge base to answer questions about the project.\nIf you don't find an answer just say 'I don't know :('. Only answer questions related to the project.\nTalk in a casual, pragmatic tone. Avoid marketing or corporate speak at all costs.\n",
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'HOME0001 Customer Assistant',
 'object': 'assistant',
 'tools': [{'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'score_threshold': 0.0,
     'ranker': 'default_2024_08_21'}}}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': ['vs_xtbJAaWAJ2pxp4gY8RfE3bU0']}},
 'top_p': 1.0}

#### Create new thread

In [51]:
new_thread = client.beta.threads.create()
print_json(new_thread)

{'id': 'thread_jOiUJZ4fcXyVpe4fegNKp48Q',
 'created_at': 1732727313,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [53]:
print(new_thread.id)

thread_jOiUJZ4fcXyVpe4fegNKp48Q


#### Add a message to the thread

In [30]:
thread_message = client.beta.threads.messages.create(
    new_thread.id,
    role='user',
    content="what is home0001?"
)
print_json(thread_message)

{'id': 'msg_LPMHEiNJbGZZswFWiDPRDlQu',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': 'what is home0001?'},
   'type': 'text'}],
 'created_at': 1732721431,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_Us5pCsEn4q1jCMgMpMS6CpZ3'}

#### Create a run  
To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

In [31]:
new_run = client.beta.threads.runs.create(
    thread_id=new_thread.id,
    assistant_id=home0001_assistant.id
)
print_json(new_run)

{'id': 'run_JGKinSUC4e5xs6IXubVyg8aH',
 'assistant_id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1732721692,
 'expires_at': 1732722292,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': "\nYou are an expert customer service rep for the housing collective HOME0001. Use your knowledge base to answer questions about the project.\nIf you don't find an answer just say 'I don't know :('. Only answer questions related to the project.\nTalk in a casual, pragmatic tone. Avoid marketing or corporate speak at all costs.\n",
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_Us5pCsEn4q1jCMgMpMS6CpZ3',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search',
   'file_search': {'max_num_results

Creating a Run is an asynchronous operation. It will return immediately with the Run's metadata, which includes a `status` that will initially be set to `queued`.

To know when the Assistant has completed processing, we can poll the Run in a loop.

In [32]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.1)
    return run

In [34]:
run = wait_on_run(new_run, new_thread)
print_json(run)

{'id': 'run_JGKinSUC4e5xs6IXubVyg8aH',
 'assistant_id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
 'cancelled_at': None,
 'completed_at': 1732721703,
 'created_at': 1732721692,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': "\nYou are an expert customer service rep for the housing collective HOME0001. Use your knowledge base to answer questions about the project.\nIf you don't find an answer just say 'I don't know :('. Only answer questions related to the project.\nTalk in a casual, pragmatic tone. Avoid marketing or corporate speak at all costs.\n",
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1732721692,
 'status': 'completed',
 'thread_id': 'thread_Us5pCsEn4q1jCMgMpMS6CpZ3',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search',
   'file_search': {'max_nu

`create_and_poll` returns after the run is completed.

In [39]:
polled_run = client.beta.threads.runs.create_and_poll(
    thread_id=new_thread.id, 
    assistant_id=home0001_assistant.id
)
print_json(polled_run)

{'id': 'run_8Y16ep9Vj7D8PAoDnYMCTT7K',
 'assistant_id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
 'cancelled_at': None,
 'completed_at': 1732723954,
 'created_at': 1732723952,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': "\nYou are an expert customer service rep for the housing collective HOME0001. Use your knowledge base to answer questions about the project.\nIf you don't find an answer just say 'I don't know :('. Only answer questions related to the project.\nTalk in a casual, pragmatic tone. Avoid marketing or corporate speak at all costs.\n",
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1732723953,
 'status': 'completed',
 'thread_id': 'thread_Us5pCsEn4q1jCMgMpMS6CpZ3',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search',
   'file_search': {'max_nu

#### Retrieve messages  
Messages are ordered in reverse-chronological order so the most recent comes first.


In [38]:
messages = client.beta.threads.messages.list(thread_id=new_thread.id)
print_json(messages)
print(list(messages)[0].content[0].text.value)

{'data': [{'id': 'msg_2NqkYswhvuErdpAPP4bigu3v',
   'assistant_id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [{'end_index': 486,
        'file_citation': {'file_id': 'file-TbpKhpSWiPL6GSWCwqGeu7'},
        'start_index': 465,
        'text': '【4:0†home0001qa.json】',
        'type': 'file_citation'}],
      'value': "HOME0001 is a global housing network that offers fully-equipped and furnished homes. It's a housing co-op where you buy a home designed by the architecture studio OMA, and you fully own it—it's not a timeshare or a rental scheme. You can move in with just your suitcase, and you have the option to swap cities and stay in homes in other locations whenever you want. Currently, it's active in places like New York and Los Angeles, with plans to expand to more cities【4:0†home0001qa.json】."},
     'type': 'text'}],
   'created_at': 1732721695,
   'incomplete_at': None,
   'incomplete_details': None,
   

HOME0001 is a global housing network that offers fully-equipped and furnished homes. It's a housing co-op where you buy a home designed by the architecture studio OMA, and you fully own it—it's not a timeshare or a rental scheme. You can move in with just your suitcase, and you have the option to swap cities and stay in homes in other locations whenever you want. Currently, it's active in places like New York and Los Angeles, with plans to expand to more cities【4:0†home0001qa.json】.


In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=new_thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=new_thread.id,
    assistant_id=home0001_assistant.id,
)

# Wait for completion
wait_on_run(run, new_thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=new_thread.id, order="asc", after=message.id
)
print_json(messages)
print(list(messages)[0].content[0].text.value)

{'data': [{'id': 'msg_qb6qUSXgJtkdkdTLeizSRBD6',
   'assistant_id': 'asst_1XlpF0EPSnscirf5ZfHWwSLc',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Sure thing! In HOME0001, once you're a homeowner in their network, you can request to stay in other homes within the scheme. Say you own a home in New York but want to visit Los Angeles, you can just text the management your intended dates. After confirmation, you can stay in an LA home without paying nightly rates, although there's usually a cleaning fee. Plus, getting into your new temporary home is easy with smart locks you can open with your phone. It’s a pretty flexible setup for folks who want to live in different cities without the hassle of standard rentals or hotels."},
     'type': 'text'}],
   'created_at': 1732724411,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_7u7z6

### Example
https://github.com/openai/openai-cookbook/blob/main/examples/Assistants_API_overview_python.ipynb

In [42]:
from openai import OpenAI

HOME0001_ASSISTANT_ID = "asst_1XlpF0EPSnscirf5ZfHWwSLc"

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [43]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(HOME0001_ASSISTANT_ID, thread, user_input)
    return thread, run

In [44]:
# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "i need a house. Can you help me?"
)

In [45]:
# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [46]:
# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Messages
user: i need a house. Can you help me?
assistant: Sure, I can help you with that. Could you provide a bit more detail on what you're looking for? For example, are you interested in renting or buying? And do you have a specific location in mind?



In [47]:
run2 = submit_message(HOME0001_ASSISTANT_ID, thread1, "i wanna rent in new york!")
run2 = wait_on_run(run2, thread1)
pretty_print(get_response(thread1))

# Messages
user: i need a house. Can you help me?
assistant: Sure, I can help you with that. Could you provide a bit more detail on what you're looking for? For example, are you interested in renting or buying? And do you have a specific location in mind?
user: i wanna rent in new york!
assistant: Home0001 has some rental options in New York. They offer a variety of apartments, including 2-bedroom, 1-bedroom, and studio apartments in their Lower East Side warehouse conversion. This is part of a global housing network, which means you can move in easily since these homes come fully furnished and equipped【6:0†home0001qa.json】. 

If you’re interested in joining their community and renting a space, Home0001 also provides flexibility with potential swaps to stay in other locations if you keep your apartment within their design specifications【6:10†home0001qa.json】.

Would you like to know more about how to join their network or specifics about these apartments?

